# Transfermarkt Futbol Veri Analizi

Bu notebook, Transfermarkt API'sini kullanarak kapsamlı futbol veri analizi yapar. 

## 🎯 Amaçlar
- Oyuncu market değeri analizi
- Transfer pattern analizi  
- Takım performans metrikleri
- Makine öğrenmesi ile market değeri tahmini

## 📊 Veri Kaynağı
[Transfermarkt API](https://transfermarkt-api.fly.dev/) - Futbol oyuncuları, kulüpler ve transfer verileri

## 🚀 Kullanılan Teknolojiler
- **Veri Toplama**: API istekleri, pandas
- **Analiz**: NumPy, pandas, scipy
- **Görselleştirme**: Matplotlib, Seaborn, Plotly
- **Makine Öğrenmesi**: Scikit-learn, XGBoost

## 1. Gerekli Kütüphaneleri Import Etme

Analiz için gerekli tüm kütüphaneleri import ediyoruz.

In [ ]:
# Temel veri işleme kütüphaneleri
import pandas as pd
import numpy as np
import json
import os
import sys
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme kütüphaneleri
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Makine öğrenmesi kütüphaneleri
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb

# İstatistik kütüphaneleri
from scipy import stats
from scipy.stats import pearsonr, spearmanr

# Progress bar
from tqdm import tqdm

# Kendi modüllerimizi import et
sys.path.append('../src')
from data_collection import TransfermarktAPIClient, DataCollector, get_popular_players, get_top_leagues
from analysis import PlayerAnalyzer, TransferAnalyzer, ClubAnalyzer
from visualization import PlayerVisualizer, TransferVisualizer
from models import MarketValuePredictor, TransferValueAnalyzer

# Matplotlib ayarları
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")
sns.set_palette("husl")

print("✅ Tüm kütüphaneler başarıyla import edildi!")
print(f"📅 Notebook çalışma zamanı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Veri Toplama (API'den)

Transfermarkt API'sini kullanarak oyuncu verilerini topluyoruz.

In [ ]:
# API istemcisini başlat
client = TransfermarktAPIClient()
data_collector = DataCollector(client, output_dir="../data")

print("🔌 API istemcisi başarıyla kuruldu")
print(f"📡 API Base URL: {client.config.base_url}")

# Test API bağlantısı
try:
    test_result = client.search_players("messi", page=1)
    print("✅ API bağlantısı başarılı!")
    if 'results' in test_result and test_result['results']:
        print(f"📊 Test arama sonucu: {len(test_result['results'])} oyuncu bulundu")
    else:
        print("⚠️ Test aramada sonuç bulunamadı")
except Exception as e:
    print(f"❌ API bağlantı hatası: {e}")
    print("💡 İnternet bağlantınızı kontrol edin veya API durumunu doğrulayın")

In [ ]:
# Popüler oyuncular listesi
popular_players = get_popular_players()[:10]  # İlk 10 oyuncu ile başlayalım
print("🌟 Analiz edilecek popüler oyuncular:")
for i, player in enumerate(popular_players, 1):
    print(f"  {i}. {player}")

# Veri toplama (bu işlem birkaç dakika sürebilir)
print("\n📥 Oyuncu verileri toplanıyor...")
print("⏳ Bu işlem birkaç dakika sürebilir, lütfen bekleyin...")

try:
    players_data = data_collector.collect_top_players_data(
        player_names=popular_players,
        max_results_per_search=2  # Her arama için maksimum 2 sonuç
    )
    
    print(f"✅ Veri toplama tamamlandı!")
    print(f"📊 Toplanan oyuncu sayısı: {len(players_data)}")
    
    # Veriyi bir dosyaya kaydet
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    data_file = f"../data/players_data_{timestamp}.json"
    
    with open(data_file, 'w', encoding='utf-8') as f:
        json.dump(players_data, f, ensure_ascii=False, indent=2, default=str)
    
    print(f"💾 Veriler {data_file} dosyasına kaydedildi")
    
    # Hızlı özet
    profile_count = sum(1 for p in players_data.values() if 'profile' in p)
    market_value_count = sum(1 for p in players_data.values() if 'market_value' in p)
    transfer_count = sum(1 for p in players_data.values() if 'transfers' in p)
    
    print(f"\n📈 Veri Özeti:")
    print(f"  • Profil bilgisi olan oyuncular: {profile_count}")
    print(f"  • Market değeri verisi olan oyuncular: {market_value_count}")
    print(f"  • Transfer verisi olan oyuncular: {transfer_count}")
    
except Exception as e:
    print(f"❌ Veri toplama hatası: {e}")
    print("💡 Daha sonra önceden kaydedilmiş veri dosyasını kullanabilirsiniz")
    players_data = {}  # Boş dictionary oluştur

In [ ]:
# Alternatif: Önceden kaydedilmiş veri dosyasını yükle
# (Eğer yukarıdaki veri toplama başarısız olduysa)

data_dir = "../data"
if not players_data:  # Eğer veri toplama başarısız olduysa
    print("🔄 Önceden kaydedilmiş veri dosyası aranıyor...")
    
    # Data dizinindeki JSON dosyalarını bul
    json_files = [f for f in os.listdir(data_dir) if f.startswith('players_data_') and f.endswith('.json')]
    
    if json_files:
        # En son dosyayı al
        latest_file = sorted(json_files)[-1]
        file_path = os.path.join(data_dir, latest_file)
        
        print(f"📂 Dosya bulundu: {latest_file}")
        
        with open(file_path, 'r', encoding='utf-8') as f:
            players_data = json.load(f)
        
        print(f"✅ Veri dosyası yüklendi: {len(players_data)} oyuncu")
    else:
        print("⚠️ Kaydedilmiş veri dosyası bulunamadı")
        print("💡 Önce yukarıdaki veri toplama hücresini çalıştırın")

# Veri kontrolü
if players_data:
    print(f"\n🎯 Analiz için hazır: {len(players_data)} oyuncu verisi")
    
    # İlk oyuncunun verilerini incele
    first_player_id = list(players_data.keys())[0]
    first_player = players_data[first_player_id]
    
    print(f"\n🔍 Örnek oyuncu verisi (ID: {first_player_id}):")
    print(f"  • Mevcut veri tipleri: {list(first_player.keys())}")
    
    if 'profile' in first_player:
        profile = first_player['profile']
        print(f"  • Oyuncu adı: {profile.get('name', 'N/A')}")
        print(f"  • Yaş: {profile.get('age', 'N/A')}")
        print(f"  • Pozisyon: {profile.get('position', {}).get('main', 'N/A')}")
        print(f"  • Kulüp: {profile.get('club', {}).get('name', 'N/A')}")
else:
    print("❌ Analiz için veri bulunamadı")
    print("💡 Lütfen önce veri toplama işlemini tamamlayın")

## 3. Veri Keşfi ve İlk Analiz

Toplanan verileri analiz ederek temel istatistikleri ve trendleri keşfediyoruz.

In [ ]:
# Oyuncu analiz sınıfını oluştur
if players_data:
    player_analyzer = PlayerAnalyzer(players_data)
    players_df = player_analyzer.players_df
    
    print("📊 OYUNCU VERİ ANALİZİ RAPORU")
    print("=" * 50)
    
    # Temel istatistikler
    print(f"\n📈 Temel İstatistikler:")
    print(f"  • Toplam oyuncu sayısı: {len(players_df)}")
    print(f"  • Yaş verisi olan oyuncular: {players_df['age'].notna().sum()}")
    print(f"  • Market değeri olan oyuncular: {players_df['current_market_value'].notna().sum()}")
    print(f"  • Pozisyon bilgisi olan oyuncular: {players_df['position'].notna().sum()}")
    
    # DataFrame'i görüntüle
    print(f"\n📋 Oyuncu DataFrame Özeti:")
    print(players_df.info())
    
    # İlk birkaç satırı göster
    print(f"\n🔍 İlk 5 Oyuncu:")
    display(players_df.head())
    
    # Temel istatistikler
    if players_df['current_market_value'].notna().any():
        print(f"\n💰 Market Değeri İstatistikleri (€M):")
        mv_stats = players_df['current_market_value'].describe()
        print(mv_stats)
        
        # En değerli oyuncular
        print(f"\n🌟 En Değerli 5 Oyuncu:")
        top_players = player_analyzer.get_top_players_by_value(n=5)
        display(top_players[['name', 'current_market_value', 'age', 'position', 'current_club']])
    
    # Yaş analizi
    if players_df['age'].notna().any():
        print(f"\n🎂 Yaş Analizi:")
        age_analysis = player_analyzer.analyze_age_distribution()
        print(f"  • Ortalama yaş: {age_analysis['statistics']['mean']:.1f}")
        print(f"  • Median yaş: {age_analysis['statistics']['50%']:.1f}")
        print(f"  • En genç oyuncu: {age_analysis['statistics']['min']:.0f} yaş")
        print(f"  • En yaşlı oyuncu: {age_analysis['statistics']['max']:.0f} yaş")
        
        print(f"\n👥 Yaş Grupları:")
        for group, count in age_analysis['age_ranges'].items():
            print(f"  • {group}: {count} oyuncu")
    
    # Pozisyon dağılımı
    if players_df['position'].notna().any():
        print(f"\n⚽ Pozisyon Dağılımı:")
        position_dist = player_analyzer.analyze_position_distribution()
        for pos, count in position_dist.head().items():
            print(f"  • {pos}: {count} oyuncu")
    
    # Uyruk dağılımı
    if players_df['nationality'].notna().any():
        print(f"\n🌍 En Yaygın Uyruklar:")
        nationality_dist = player_analyzer.analyze_nationality_distribution()
        for nat, count in nationality_dist.head().items():
            print(f"  • {nat}: {count} oyuncu")

else:
    print("❌ Analiz için veri bulunamadı")

In [ ]:
# Transfer Analizi
if players_data:
    print("\n🔄 TRANSFER ANALİZİ")
    print("=" * 50)
    
    # Transfer analiz sınıfını oluştur
    transfer_analyzer = TransferAnalyzer(players_data)
    transfers_df = transfer_analyzer.transfers_df
    
    if not transfers_df.empty:
        print(f"📊 Transfer Verisi Özeti:")
        print(f"  • Toplam transfer kaydı: {len(transfers_df)}")
        print(f"  • Ücretli transferler: {len(transfers_df[transfers_df['fee'] > 0])}")
        print(f"  • Ücretsiz transferler: {len(transfers_df[transfers_df['fee'] == 0])}")
        print(f"  • Gelecek transferler: {len(transfers_df[transfers_df['upcoming'] == True])}")
        
        # Transfer istatistikleri
        transfer_stats = transfer_analyzer.get_transfer_statistics()
        
        if 'total_transfer_volume' in transfer_stats:
            print(f"\n💰 Transfer Finansal Özeti:")
            print(f"  • Toplam transfer hacmi: €{transfer_stats['total_transfer_volume']:.1f}M")
            print(f"  • Ortalama transfer ücreti: €{transfer_stats['average_transfer_fee']:.1f}M")
            print(f"  • En pahalı transfer: €{transfer_stats['highest_transfer_fee']:.1f}M")
            
            if transfer_stats['most_expensive_transfer']:
                expensive = transfer_stats['most_expensive_transfer']
                print(f"    → {expensive['player']} ({expensive['from_club']} → {expensive['to_club']})")
        
        # En aktif kulüpler
        print(f"\n🏟️ En Aktif Kulüpler (Alım):")
        active_clubs = transfer_analyzer.analyze_most_active_clubs(direction='buying')
        if not active_clubs.empty:
            for i, (_, row) in enumerate(active_clubs.head().iterrows()):
                print(f"  {i+1}. {row['to_club_name']}: €{row['total_spent']:.1f}M harcama")
        
        # Yıllık transfer trendleri
        yearly_trends = transfer_analyzer.analyze_transfer_trends_by_year()
        if not yearly_trends.empty and len(yearly_trends) > 1:
            print(f"\n📈 Transfer Trendleri:")
            recent_years = yearly_trends.tail(3)
            for _, year_data in recent_years.iterrows():
                print(f"  • {year_data['year']}: {year_data['transfer_count']} transfer, €{year_data['total_volume']:.1f}M hacim")
    
    else:
        print("⚠️ Transfer verisi bulunamadı")
        print("💡 Veri toplama işlemi transfer verilerini içermeyebilir")

else:
    print("❌ Transfer analizi için veri bulunamadı")

## 4. Veri Görselleştirme

Analiz sonuçlarını görsel grafiklerle daha anlaşılır hale getiriyoruz.

In [ ]:
# Oyuncu Görselleştirmeleri
if players_data and not players_df.empty:
    print("🎨 Görselleştirmeler oluşturuluyor...")
    
    # Görselleştirme sınıfını oluştur
    player_visualizer = PlayerVisualizer(players_df)
    
    # 1. Market değeri dağılımı
    if players_df['current_market_value'].notna().any():
        print("\n💰 Market Değeri Analizi Grafikleri")
        fig = player_visualizer.plot_market_value_distribution(figsize=(15, 10))
        plt.show()
    
    # 2. Yaş dağılımı
    if players_df['age'].notna().any():
        print("\n🎂 Yaş Dağılımı Grafikleri")
        fig = player_visualizer.plot_age_distribution(figsize=(12, 6))
        plt.show()
    
    # 3. Pozisyon analizi
    if players_df['position'].notna().any():
        print("\n⚽ Pozisyon Analizi Grafikleri")
        fig = player_visualizer.plot_position_analysis(figsize=(14, 8))
        plt.show()
    
    # 4. İnteraktif scatter plot (Yaş vs Market Değeri)
    if players_df['age'].notna().any() and players_df['current_market_value'].notna().any():
        print("\n📊 İnteraktif Yaş vs Market Değeri Grafiği")
        fig = player_visualizer.create_interactive_market_value_plot()
        fig.show()

else:
    print("❌ Görselleştirme için yeterli veri yok")

In [ ]:
# Transfer Görselleştirmeleri
if players_data and 'transfers_df' in locals() and not transfers_df.empty:
    print("\n🔄 Transfer Grafikleri")
    
    # Transfer görselleştirme sınıfını oluştur
    transfer_visualizer = TransferVisualizer(transfers_df)
    
    # Transfer trendleri
    if len(transfers_df[transfers_df['date'].notna()]) > 0:
        print("\n📈 Transfer Trendleri")
        fig = transfer_visualizer.plot_transfer_trends(figsize=(15, 10))
        plt.show()
    
    # Kulüp aktiviteleri
    paid_transfers = transfers_df[(transfers_df['fee'] > 0) & (transfers_df['upcoming'] == False)]
    if len(paid_transfers) > 0:
        print("\n🏟️ Kulüp Transfer Aktiviteleri")
        fig = transfer_visualizer.plot_club_activity(figsize=(15, 8))
        plt.show()
    
    # İnteraktif transfer timeline
    if len(paid_transfers) > 0:
        print("\n⏰ İnteraktif Transfer Zaman Çizelgesi")
        fig = transfer_visualizer.create_interactive_transfer_timeline()
        fig.show()

else:
    print("⚠️ Transfer görselleştirmesi için yeterli veri yok")

## 5. Makine Öğrenmesi - Market Değeri Tahmini

Oyuncu özelliklerini kullanarak market değerlerini tahmin eden modeller geliştiriyoruz.

In [ ]:
# Market Değeri Tahmin Modeli
if players_data and not players_df.empty:
    print("🤖 MARKET DEĞERİ TAHMİN MODELİ")
    print("=" * 50)
    
    # Market değeri olan oyuncuları kontrol et
    mv_data = players_df.dropna(subset=['current_market_value'])
    
    if len(mv_data) >= 10:  # En az 10 örnek gerekli
        print(f"📊 Model eğitimi için {len(mv_data)} oyuncu verisi mevcut")
        
        # Market değeri tahmin modeli oluştur
        predictor = MarketValuePredictor()
        
        print("\n⚙️ Model eğitimi başlıyor...")
        print("⏳ Bu işlem birkaç dakika sürebilir...")
        
        try:
            # Modeli eğit
            results = predictor.train_model(players_df, test_size=0.2, random_state=42)
            
            if results:
                print("✅ Model eğitimi tamamlandı!")
                print(f"\n🏆 En İyi Model: {results['best_model']}")
                print(f"📈 Test R² Skoru: {results['best_score']:.4f}")
                
                # Tüm model sonuçlarını göster
                print(f"\n📊 Model Karşılaştırması:")
                print("-" * 80)
                print(f"{'Model':<20} {'Test R²':<10} {'Test RMSE':<12} {'Test MAE':<10}")
                print("-" * 80)
                
                for model_name, metrics in results['all_results'].items():
                    print(f"{model_name:<20} {metrics['test_r2']:<10.4f} {metrics['test_rmse']:<12.2f} {metrics['test_mae']:<10.2f}")
                
                # Feature importance
                importance_df = predictor.get_feature_importance()
                if importance_df is not None and not importance_df.empty:
                    print(f"\n🎯 En Önemli Özellikler:")
                    top_features = importance_df.head(10)
                    for i, (_, row) in enumerate(top_features.iterrows(), 1):
                        print(f"  {i:2d}. {row['feature']}: {row['importance']:.4f}")
                
                # Model performance görselleştirmesi
                print(f"\n📈 Model Performans Karşılaştırması")
                
                model_names = list(results['all_results'].keys())
                test_r2_scores = [results['all_results'][name]['test_r2'] for name in model_names]
                test_rmse_scores = [results['all_results'][name]['test_rmse'] for name in model_names]
                
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
                
                # R² skorları
                bars1 = ax1.bar(model_names, test_r2_scores, color='skyblue', alpha=0.7)
                ax1.set_title('Model R² Skorları (Test Seti)', fontsize=14, fontweight='bold')
                ax1.set_ylabel('R² Skoru')
                ax1.set_ylim(0, 1)
                ax1.tick_params(axis='x', rotation=45)
                
                # Değerleri barların üzerine ekle
                for bar, score in zip(bars1, test_r2_scores):
                    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                            f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
                
                # RMSE skorları
                bars2 = ax2.bar(model_names, test_rmse_scores, color='lightcoral', alpha=0.7)
                ax2.set_title('Model RMSE Skorları (Test Seti)', fontsize=14, fontweight='bold')
                ax2.set_ylabel('RMSE (€M)')
                ax2.tick_params(axis='x', rotation=45)
                
                # Değerleri barların üzerine ekle
                for bar, score in zip(bars2, test_rmse_scores):
                    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(test_rmse_scores)*0.01, 
                            f'{score:.1f}', ha='center', va='bottom', fontweight='bold')
                
                plt.tight_layout()
                plt.show()
                
                # Modeli kaydet
                model_path = "../models/market_value_predictor.pkl"
                os.makedirs("../models", exist_ok=True)
                predictor.save_model(model_path)
                print(f"\n💾 Model {model_path} dosyasına kaydedildi")
                
            else:
                print("❌ Model eğitimi başarısız oldu")
                
        except Exception as e:
            print(f"❌ Model eğitimi hatası: {e}")
            print("💡 Veri kalitesini kontrol edin")
    
    else:
        print(f"⚠️ Model eğitimi için yeterli veri yok")
        print(f"📊 Mevcut market değeri verisi: {len(mv_data)} oyuncu")
        print("💡 En az 10 oyuncu verisi gerekli")

else:
    print("❌ Model eğitimi için veri bulunamadı")

## 6. Sonuçlar ve Bulgular

### 📊 Analiz Özeti

Bu notebook'ta gerçekleştirilen analiz sonuçlarının özeti:

1. **Veri Toplama**: Transfermarkt API'sinden popüler oyuncular için veri toplama
2. **Veri Keşfi**: Temel istatistikler, dağılımlar ve trendler analizi
3. **Görselleştirme**: Market değeri, yaş, pozisyon ve transfer analizleri
4. **Makine Öğrenmesi**: Market değeri tahmin modeli geliştirme

### 🔍 Ana Bulgular

- **Market Değeri Trendleri**: En değerli oyuncuların pozisyon ve yaş dağılımları
- **Transfer Analizi**: En aktif kulüpler ve transfer hacmi trendleri
- **Model Performansı**: Oyuncu özelliklerinden market değeri tahmin başarısı

### 🚀 Gelecek Adımlar

1. **Daha Fazla Veri**: Daha geniş oyuncu ve kulüp verileri toplama
2. **İleri Analiz**: Sezonluk performans metrikleri entegrasyonu
3. **Model İyileştirme**: Daha sofistike özellik mühendisliği
4. **Dashboard**: İnteraktif analiz dashboard'u geliştirme

### 💡 Öneriler

- Düzenli veri güncellemeleri için otomatik pipeline kurulması
- Farklı ligler için karşılaştırmalı analizler
- Transfer başarı tahmini modeli geliştirme
- Oyuncu performans metrikleri ile market değeri korelasyonu